In [1]:
import os
os.chdir("../")

import semiolog as slg

In [2]:
from pyinstrument import Profiler
import sys

In [3]:
semiotic = slg.Cenematic("fr_wiki",requested_cpu=4)

Vocabulary will not be loaded from file.



In [7]:
(92+8) % 10 == 0

True

In [4]:
semiotic.vocab.build_new(vocab_size = -100, parallel=False, save=True, save_step=10)

Pair: init, ---:   0%|          | 0/100 [00:00<?, ?it/s]

Alphabet Size: 493


/usr/local/lib/python3.9/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])
Pair: ('v', 'er'), 19844: 100%|██████████| 100/100 [04:59<00:00,  3.00s/it]

Vocabulary built
Vocabulary saved to models/fr_wiki/vocabulary


In [4]:
import regex as re
import numpy as np
from collections import Counter
from scipy.sparse import csr_matrix, lil_matrix, coo_matrix
from tqdm.notebook import tqdm, trange
from functools import partial
from functools import reduce
import operator

import time
from pyinstrument import Profiler
import sys

from temp import findall_contexts, findall_contexts_list, find_best_pair, agglutinate_list

In [5]:
def build_nb(
    corpus = None,
    voc_final_length = -30,
    # save = False,
    # save_step = None,
    # progress_bar = True,
    # resume_merges = False,
    parallel = False,
    sparse = True,
    sparse_mode = "csr",
    cpu_count = 4,
    corpus_length = None,
    normalizer = None,
):
    def agglutinate_chain(pair, cl_chain):
        bigram = re.escape(" ".join(pair))
        p = re.compile(r"(?<!\S)" + bigram + r"(?!\S)")
        cl_chain = p.sub("".join(pair), cl_chain)
        return cl_chain

    def extract_drc(pairs, encoder: dict):
        data = []
        rows = []
        columns = []
        for (r,c),d in pairs:
            data.append(d)
            rows.append(encoder[r])
            columns.append(encoder[c])
        return data, rows, columns

    def parallel_chain(chain, n_of_parts, overlap = 0):
        """
        Breaks the chain in n chunks to compute best pair of terms. Chunks are overlapping by one term, so as no pair of terms is lost due to the break.
        """
        if not isinstance(chain,list):
            chain = list(chain)
        chunk_size = int(len(chain) / n_of_parts)+1
        for i in range(0, len(chain), chunk_size):
            yield chain[i : i + chunk_size + overlap]

    def separate_chain(chain, n_of_parts, best_pair: list):
        """
        Separate a chain (in list form) for parallel processing of regex findall of pair, taking care that the cuts of the chunks don't fall in the neiborhood of the pair, affecting the final counts
        """
        chunk_size = int(len(chain) / n_of_parts)+1
        b = 0
        n = chunk_size
        chain_len = len(chain)
        for i in range(n_of_parts):
            n = (i+1)*chunk_size
            if chain_len > n:
                while chain[n-2:n] == best_pair or chain[n-1:n+1] == best_pair:
                    n = n+1
            yield ("[SEP_i] " if i!=0 else "") + " ".join(chain[b:n]) + (" [SEP_i]" if i!=n_of_parts-1 else "")
            b = n-1
        
        
    # normalizer = eval(f"slg.syntagmatic.tokenizer.normalizers.Sequence({semiotic.config.vocabulary.normalizer})")
    
    if parallel:
        
        par_corpus = parallel_chain(corpus[:corpus_length], cpu_count)

        result = slg.util.multiprocessing_tqdm(partial(semiotic.vocab.chain_list_alpha, normalizer), par_corpus, cores=cpu_count, desc="Normalize & Alphabet")
        
        chain_list = []
        alphabet = Counter()
        for chain_l, alpha in result:
            chain_list += chain_l
            alphabet += alpha
            
    else:
        chain_list, alphabet = semiotic.vocab.chain_list_alpha(normalizer, semiotic.corpus.train[:corpus_length], progress_bar=True)

    # cl, alphabet = semiotic.vocab.chain_list_alpha(normalizer, semiotic.corpus.train, progress_bar=True)
    cl_chain = "[SEP] "+" ".join(chain_list)+" [SEP]"
    encode = {k:i for i,(k,v) in enumerate(alphabet.most_common())}
    decode = {i:k for k,i in encode.items()}
    new_i = len(encode)
    if parallel:
        
        par_chain = parallel_chain(chain_list, cpu_count, overlap=1)
        
        result = slg.util.multiprocessing(find_best_pair, par_chain, cores=cpu_count) 
                            
        pairs = reduce(operator.add, result)
        pairs = pairs.most_common()
        
    else:
        pairs = find_best_pair(chain_list).most_common()
    if voc_final_length<0:
        voc_final_length = new_i + abs(voc_final_length)
        
    if sparse:
        data, rows, columns = extract_drc(pairs,encode)
        voc_matrix = coo_matrix((np.array(data), (np.array(rows),np.array(columns))), shape=(voc_final_length, voc_final_length), dtype=int)

    else:
        voc_matrix = np.zeros((voc_final_length, voc_final_length), dtype=int)
        for (row,column),value in pairs:
            voc_matrix[encode[row], encode[column]] = value
    merges = []
    delta_voc = voc_final_length - new_i
    best_pair = "init"
    pair_count = "---"

    t = trange(delta_voc) #, disable = not progress_bar)

    for _ in t:
        t.set_description(f"Pair: {best_pair}, {pair_count}")
        t.refresh()

        if sparse:
            max_i = voc_matrix.data.argmax()
            pair_row = voc_matrix.row[max_i]
            pair_col = voc_matrix.col[max_i]
            pair_count = voc_matrix.data[max_i]
        else:
            pair_row,pair_col = np.unravel_index(np.argmax(voc_matrix, axis=None), voc_matrix.shape)
            pair_count = voc_matrix[pair_row,pair_col]
        
        if pair_count == 0:
            break

        best_pair = (decode[pair_row], decode[pair_col])
        best_pair_string = " ".join(best_pair)
        merges.append(best_pair_string)
        best_pair_string_voc = "".join(best_pair)
        re_voc_l = "("+"|".join([" "+k+" " for k in encode.keys()]+["\[SEP\] ","\[SEP_i\] "])+")"
        re_voc_r = "("+"|".join([" "+k+" " for k in encode.keys()]+[" \[SEP\]"," \[SEP_i\]"])+")"
        if parallel:
            result = slg.util.multiprocessing(
                partial(findall_contexts,best_pair_string=best_pair_string,re_voc_l=re_voc_l,re_voc_r=re_voc_r),
                separate_chain(cl_chain.split(), cpu_count, list(best_pair)),
                cores = cpu_count
                )
            merge_context = reduce(operator.add, result)
        else:
            merge_context = re.findall(re_voc_l+best_pair_string+re_voc_r, cl_chain, overlapped=True)
        merge_context_count_l = Counter()
        merge_context_count_r = Counter()
        for l,r in merge_context:
            if "[SEP]" not in l:
                merge_context_count_l[encode[l.strip()]] += 1
            if "[SEP]" not in r:
                merge_context_count_r[encode[r.strip()]] += 1
        
        if sparse:
            # Convert matrix to CSR or LIL, for item attribution and arithmetic 
            if sparse_mode == "csr":
                voc_matrix = voc_matrix.tocsr()
            else:
                voc_matrix = voc_matrix.tolil()
        
        for row,key in merge_context_count_l.items():
            voc_matrix[row,new_i] = key
            
        for column,key in merge_context_count_r.items():
            voc_matrix[new_i,column] = key

        # Correct previous counts
        
        # compute #(l,r)-(l,r)
        pair_pair_count = len(re.findall(" "+best_pair_string+" "+best_pair_string+" ", cl_chain, overlapped=False))
        # remove #(l,r)-(l,r) from (l,r)-l
        voc_matrix[new_i,pair_row] -= pair_pair_count
        # remove #(l,r)-(l,r) from r-(l,r)
        voc_matrix[pair_col,new_i] -= pair_pair_count
        # remove #(l,r)-(l,r) from r-l
        voc_matrix[pair_col,pair_row] -= pair_pair_count
        # substract (l,r)- from r-
        voc_matrix[pair_col,:new_i] -= voc_matrix[new_i,:new_i]
        # substract -(l,r)- from -l
        voc_matrix[:new_i,pair_row] -= voc_matrix[:new_i,new_i]
        
        # set l-r to 0
        voc_matrix[pair_row,pair_col] = 0
        # register #(l,r)-(l,r)
        voc_matrix[new_i,new_i] = pair_pair_count
        
        if sparse:
            # Convert matrix back to COO, to restart the loop
            voc_matrix = voc_matrix.tocoo()
        
        best_pair_string_voc = "".join(best_pair)
        encode[best_pair_string_voc] = new_i
        decode[new_i] = best_pair_string_voc
        new_i += 1
        cl_chain = agglutinate_chain(best_pair_string.split(),cl_chain)


    if sparse:
        freq_values = voc_matrix.sum(axis=1).T.tolist()[0]
    else:
        freq_values = voc_matrix.sum(axis=1).T.tolist()
    vocabulary = {decode[i]:v for i,v in enumerate(freq_values) if v>0} # Make sure dimension of matrix and size of voc coincide
    vocabulary = sorted(vocabulary.items(), key=lambda x: x[1], reverse=True)
    
    return merges, vocabulary

In [6]:
def build_nb_new(
    corpus = None,
    voc_final_length = -30,
    # save = False,
    # save_step = None,
    # progress_bar = True,
    # resume_merges = False,
    parallel = False,
    sparse = True,
    sparse_mode = "csr",
    cpu_count = 4,
    corpus_length = None,
    normalizer = None,
):
    def agglutinate_chain(pair, cl_chain):
        bigram = re.escape(" ".join(pair))
        p = re.compile(r"(?<!\S)" + bigram + r"(?!\S)")
        cl_chain = p.sub("".join(pair), cl_chain)
        return cl_chain

    def extract_drc(pairs, encoder: dict):
        data = []
        rows = []
        columns = []
        for (r,c),d in pairs:
            data.append(d)
            rows.append(encoder[r])
            columns.append(encoder[c])
        return data, rows, columns

    def parallel_chain(chain, n_of_parts, overlap = 0):
        """
        Breaks the chain in n chunks to compute best pair of terms. Chunks are overlapping by one term, so as no pair of terms is lost due to the break.
        """
        if not isinstance(chain,list):
            chain = list(chain)
        chunk_size = int(len(chain) / n_of_parts)+1
        for i in range(0, len(chain), chunk_size):
            yield chain[i : i + chunk_size + overlap]

    def separate_chain(chain, n_of_parts, best_pair: list):
        """
        Separate a chain (in list form) for parallel processing of regex findall of pair, taking care that the cuts of the chunks don't fall in the neiborhood of the pair, affecting the final counts
        """
        chunk_size = int(len(chain) / n_of_parts)+1
        b = 0
        n = chunk_size
        chain_len = len(chain)
        for i in range(n_of_parts):
            n = (i+1)*chunk_size
            if chain_len > n:
                while chain[n-2:n] == best_pair or chain[n-1:n+1] == best_pair:
                    n = n+1
            yield chain[b:n]
            b = n-1
        
        
    # normalizer = eval(f"slg.syntagmatic.tokenizer.normalizers.Sequence({semiotic.config.vocabulary.normalizer})")
    
    if parallel:
        
        par_corpus = parallel_chain(corpus[:corpus_length], cpu_count)

        result = slg.util.multiprocessing_tqdm(partial(semiotic.vocab.chain_list_alpha, normalizer), par_corpus, cores=cpu_count, desc="Normalize & Alphabet")
        
        chain_list = []
        alphabet = Counter()
        for chain_l, alpha in result:
            chain_list += chain_l
            alphabet += alpha
            
    else:
        chain_list, alphabet = semiotic.vocab.chain_list_alpha(normalizer, semiotic.corpus.train[:corpus_length], progress_bar=True)

    # cl, alphabet = semiotic.vocab.chain_list_alpha(normalizer, semiotic.corpus.train, progress_bar=True)
    # cl_chain = "[SEP] "+" ".join(chain_list)+" [SEP]"
    cl_chain = " ".join(chain_list)
    encode = {k:i for i,(k,v) in enumerate(alphabet.most_common())}
    decode = {i:k for k,i in encode.items()}
    new_i = len(encode)
    if parallel:
        
        par_chain = parallel_chain(chain_list, cpu_count, overlap=1)
        
        result = slg.util.multiprocessing(find_best_pair, par_chain, cores=cpu_count) 
                            
        pairs = reduce(operator.add, result)
        pairs = pairs.most_common()
        
    else:
        pairs = find_best_pair(chain_list).most_common()
    if voc_final_length<0:
        voc_final_length = new_i + abs(voc_final_length)
        
    if sparse:
        data, rows, columns = extract_drc(pairs,encode)
        voc_matrix = coo_matrix((np.array(data), (np.array(rows),np.array(columns))), shape=(voc_final_length, voc_final_length), dtype=int)

    else:
        voc_matrix = np.zeros((voc_final_length, voc_final_length), dtype=int)
        for (row,column),value in pairs:
            voc_matrix[encode[row], encode[column]] = value
    merges = []
    delta_voc = voc_final_length - new_i
    best_pair = "init"
    pair_count = "---"

    t = trange(delta_voc) #, disable = not progress_bar)

    for _ in t:
        t.set_description(f"Pair: {best_pair}, {pair_count}")
        t.refresh()

        if sparse:
            max_i = voc_matrix.data.argmax()
            pair_row = voc_matrix.row[max_i]
            pair_col = voc_matrix.col[max_i]
            pair_count = voc_matrix.data[max_i]
        else:
            pair_row,pair_col = np.unravel_index(np.argmax(voc_matrix, axis=None), voc_matrix.shape)
            pair_count = voc_matrix[pair_row,pair_col]
        
        if pair_count == 0:
            break

        best_pair = (decode[pair_row], decode[pair_col])
        best_pair_string = " ".join(best_pair)
        merges.append(best_pair_string)
        best_pair_string_voc = "".join(best_pair)
        # re_voc_l = "("+"|".join([" "+k+" " for k in encode.keys()]+["\[SEP\] ","\[SEP_i\] "])+")"
        # re_voc_r = "("+"|".join([" "+k+" " for k in encode.keys()]+[" \[SEP\]"," \[SEP_i\]"])+")"
        if parallel:
            result = slg.util.multiprocessing(
                partial(findall_contexts_list,best_pair=best_pair,encode=encode),
                separate_chain(chain_list, cpu_count, list(best_pair)),
                cores = cpu_count
                )
            merge_context_count_l = Counter()
            merge_context_count_r = Counter()
            pair_pair_count = 0
            for merge_l, merge_r, ppcount in result:
                merge_context_count_l += merge_l
                merge_context_count_r += merge_r
                pair_pair_count += ppcount
        else:
            merge_context_count_l, merge_context_count_r, pair_pair_count = findall_contexts_list(chain_list,best_pair,encode)
        
        if sparse:
            # Convert matrix to CSR or LIL, for item attribution and arithmetic 
            if sparse_mode == "csr":
                voc_matrix = voc_matrix.tocsr()
            else:
                voc_matrix = voc_matrix.tolil()
        
        for row,key in merge_context_count_l.items():
            voc_matrix[row,new_i] = key
            
        for column,key in merge_context_count_r.items():
            voc_matrix[new_i,column] = key

        # Correct previous counts
        
        # # compute #(l,r)-(l,r)
        # pair_pair_count = len(re.findall(" "+best_pair_string+" "+best_pair_string+" ", cl_chain, overlapped=False))
        # remove #(l,r)-(l,r) from (l,r)-l
        voc_matrix[new_i,pair_row] -= pair_pair_count
        # remove #(l,r)-(l,r) from r-(l,r)
        voc_matrix[pair_col,new_i] -= pair_pair_count
        # remove #(l,r)-(l,r) from r-l
        voc_matrix[pair_col,pair_row] -= pair_pair_count
        # substract (l,r)- from r-
        voc_matrix[pair_col,:new_i] -= voc_matrix[new_i,:new_i]
        # substract -(l,r)- from -l
        voc_matrix[:new_i,pair_row] -= voc_matrix[:new_i,new_i]
        
        # set l-r to 0
        voc_matrix[pair_row,pair_col] = 0
        # register #(l,r)-(l,r)
        voc_matrix[new_i,new_i] = pair_pair_count
        
        if sparse:
            # Convert matrix back to COO, to restart the loop
            voc_matrix = voc_matrix.tocoo()
        
        best_pair_string_voc = "".join(best_pair)
        encode[best_pair_string_voc] = new_i
        decode[new_i] = best_pair_string_voc
        new_i += 1
        cl_chain = agglutinate_chain(best_pair_string.split(),cl_chain)
        chain_list = cl_chain.split()


    if sparse:
        freq_values = voc_matrix.sum(axis=1).T.tolist()[0]
    else:
        freq_values = voc_matrix.sum(axis=1).T.tolist()
    vocabulary = {decode[i]:v for i,v in enumerate(freq_values) if v>0} # Make sure dimension of matrix and size of voc coincide
    vocabulary = sorted(vocabulary.items(), key=lambda x: x[1], reverse=True)
    
    return merges, vocabulary

In [8]:
normalizer = slg.syntagmatic.tokenizer.normalizers.Sequence(semiotic.config.vocabulary.normalizer)
    
profile = Profiler()
profile.start()

merges, vocabulary = build_nb(
    semiotic.corpus.train,
    normalizer=normalizer,
    voc_final_length = -30,
    parallel=True)

profile.stop()
print(profile.output_text(unicode=True, color=True))



  _     ._   __/__   _ _  _  _ _/_   Recorded: 17:12:21  Samples:  7290
 /_//_/// /_\ / //_// / //_'/ //     Duration: 104.636   CPU time: 61.937
/   _/                      v3.4.2

Program: ipykernel_launcher --ip=127.0.0.1 --stdin=9018 --control=9016 --hb=9015 --Session.signature_scheme="hmac-sha256" --Session.key=b"94d087ff-ccf2-43bf-b73b-54902b508d73" --shell=9017 --transport="tcp" --iopub=9019 --f=/var/folders/k4/tv2m69x552l9grwtcdhc0zxm0000gn/T/tmp-13680qAjBPdIXB0BA.json

104.636 run_code  IPython/core/interactiveshell.py:3400
└─ 104.636 <module>  <ipython-input-8-9f56a99ed05b>:6
   └─ 104.563 build_nb  <ipython-input-5-fd4ee32eb418>:1
      ├─ 69.834 multiprocessing  semiolog/util.py:58
      │  ├─ 42.395 __exit__  concurrent/futures/_base.py:635
      │  │     [17 frames hidden]  concurrent, threading, <built-in>, mu...
      │  │        41.545 lock.acquire  <built-in>:0
      │  └─ 27.430 map  concurrent/futures/process.py:702
      │        [139 frames hidden]  concurrent, 

In [13]:
normalizer = slg.syntagmatic.tokenizer.normalizers.Sequence(semiotic.config.vocabulary.normalizer)
    
profile = Profiler()
profile.start()

merges_new, vocabulary_new = build_nb_new(
    semiotic.corpus.train,
    normalizer=normalizer,
    voc_final_length = -30,
    parallel=True)

profile.stop()
print(profile.output_text(unicode=True, color=True))



  _     ._   __/__   _ _  _  _ _/_   Recorded: 16:20:18  Samples:  3318
 /_//_/// /_\ / //_// / //_'/ //     Duration: 121.256   CPU time: 80.848
/   _/                      v3.4.2

Program: ipykernel_launcher --ip=127.0.0.1 --stdin=9018 --control=9016 --hb=9015 --Session.signature_scheme="hmac-sha256" --Session.key=b"4f3bce2f-170f-493c-8a3b-6232354de0d2" --shell=9017 --transport="tcp" --iopub=9019 --f=/var/folders/k4/tv2m69x552l9grwtcdhc0zxm0000gn/T/tmp-13680rVHW7F2zrHlL.json

121.255 run_code  IPython/core/interactiveshell.py:3400
└─ 121.255 <module>  <ipython-input-13-46b97c605aec>:6
   └─ 120.944 build_nb_new  <ipython-input-6-0c1cfdc702da>:1
      ├─ 79.738 multiprocessing  semiolog/util.py:58
      │  ├─ 47.981 __exit__  concurrent/futures/_base.py:635
      │  │     [12 frames hidden]  concurrent, threading, <built-in>, mu...
      │  │        47.907 lock.acquire  <built-in>:0
      │  └─ 31.724 map  concurrent/futures/process.py:702
      │        [162 frames hidden]  concurr

In [7]:
normalizer = slg.syntagmatic.tokenizer.normalizers.Sequence(semiotic.config.vocabulary.normalizer)
    
profile = Profiler()
profile.start()

merges_new, vocabulary_new = build_nb_new(
    semiotic.corpus.train,
    normalizer=normalizer,
    voc_final_length = -30,
    parallel=True)

profile.stop()
print(profile.output_text(unicode=True, color=True))

/usr/local/lib/python3.9/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])




  _     ._   __/__   _ _  _  _ _/_   Recorded: 17:28:52  Samples:  3408
 /_//_/// /_\ / //_// / //_'/ //     Duration: 127.850   CPU time: 81.221
/   _/                      v3.4.2

Program: ipykernel_launcher --ip=127.0.0.1 --stdin=9011 --control=9009 --hb=9008 --Session.signature_scheme="hmac-sha256" --Session.key=b"ce9657c1-93ff-41b6-8450-89cff71102fd" --shell=9010 --transport="tcp" --iopub=9012 --f=/var/folders/k4/tv2m69x552l9grwtcdhc0zxm0000gn/T/tmp-13680vi5S5W5Rij3d.json

127.850 run_code  IPython/core/interactiveshell.py:3400
└─ 127.850 <module>  <ipython-input-7-46b97c605aec>:6
   └─ 127.518 build_nb_new  <ipython-input-6-d180641c9b6f>:1
      ├─ 90.126 multiprocessing  semiolog/util.py:58
      │  ├─ 55.105 __exit__  concurrent/futures/_base.py:635
      │  │     [7 frames hidden]  concurrent, threading, <built-in>
      │  │        55.047 lock.acquire  <built-in>:0
      │  └─ 34.758 map  concurrent/futures/process.py:702
      │        [177 frames hidden]  concurrent, mult

In [8]:
profile.stop()
print(profile.output_text(unicode=True, color=True))


  _     ._   __/__   _ _  _  _ _/_   Recorded: 16:53:33  Samples:  2232
 /_//_/// /_\ / //_// / //_'/ //     Duration: 62.016    CPU time: 5.339
/   _/                      v3.4.2

Program: ipykernel_launcher --ip=127.0.0.1 --stdin=9018 --control=9016 --hb=9015 --Session.signature_scheme="hmac-sha256" --Session.key=b"51502a65-9e8e-47b4-9af7-0744af68f09a" --shell=9017 --transport="tcp" --iopub=9019 --f=/var/folders/k4/tv2m69x552l9grwtcdhc0zxm0000gn/T/tmp-13680WCWKxqMwEp4H.json

62.016 _run_once  asyncio/base_events.py:1815
├─ 48.866 select  selectors.py:554
│     [3 frames hidden]  selectors, <built-in>
│        48.866 kqueue.control  <built-in>:0
└─ 13.149 _run  asyncio/events.py:78
      [199 frames hidden]  asyncio, tornado, ipykernel, IPython,...
         13.143 run_code  IPython/core/interactiveshell.py:3400
         └─ 12.801 <module>  <ipython-input-7-46b97c605aec>:6
            └─ 12.801 build_nb_new  <ipython-input-6-4ed4f9368b8c>:1
               ├─ 5.471 multiprocessing_tqdm

In [9]:
print(vocabulary == vocabulary_new)
print(merges == merges_new)

True
True
